In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Grounding with Vertex AI Search

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fgrounding_with_vais.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/grounding/grounding_with_vais.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/grounding_with_vais.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| Author(s) |  [Diem Vu](https://github.com/diemtvu/) [Zhen Hu](https://github.com/undertwig/) |

## Overview

This notebook demonstrates how to use Vertex AI Search for grounding Vertex LLMs.  For more general information on grounding, see [Getting Started with Grounding with Gemini in Vertex AI](./intro-grounding-gemini.ipynb).

In this tutorial, we will cover:

* How to create a Vertex AI Search datastore with your data.
* Provide an example LLM request that uses this data for grounding.

## Get started

### Install Vertex AI SDK and other required packages

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-discoveryengine

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

[link text](https://)<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. In Colab or Colab Enterprise, you might see an error message that says "Your session crashed for an unknown reason." This is expected. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

# You also need to choose a region for your Vertex AI Search datastore.
# Vertex AI Search is a multi-region service, supporting us, eu, and global regions.
VAIS_LOCATION = "global"  # @param {type: "string"}

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

## Create Vertex AI Search engine

We will create a Vertex AI Search engine index by ingesting files from Google Cloud Storage (GCS) using the SDK. To explore additional features and UI-based instructions, see the [Vertex AI Search documentation](https://cloud.google.com/enterprise-search).

This involves 2 steps:

1. Create a datastore (corpus) and import documents: For this demo, we will use the PDF files available at `gs://cloud-samples-data/gen-app-builder/search/alphabet-sec-filings`. You can try with your own data.

2. Create `SEARCH_TIER_ENTERPRISE` engine with `SEARCH_ADD_ON_LLM` feature on top of the data store. Enterprise tier is required to get extractive answers from a search query and advanced LLM features to ensure the grounding quality.

In [ ]:
# @title Initialize the VAIS clients

import time

import google.cloud.discoveryengine_v1 as vais
from google.api_core.client_options import ClientOptions

client_options = (
    ClientOptions(api_endpoint=f"{VAIS_LOCATION}-discoveryengine.googleapis.com")
    if VAIS_LOCATION != "global"
    else None
)
data_store_service_client = vais.DataStoreServiceClient(client_options=client_options)
document_service_client = vais.DocumentServiceClient(client_options=client_options)
engine_client = vais.EngineServiceClient(client_options=client_options)


def wait_for_operation_finish(operation):
    while not operation.done:
        time.sleep(2)  # sleep 2 seconds

In [ ]:
# @title Create data store
# fmt: off
DATA_STORE_ID = "cymbal"  # @param {type:"string"}
# fmt: on

# Create a generic search data store with unstructured data content.
data_store = vais.DataStore(
    display_name="Data Store for Vertex LLM Grounding demo",
    industry_vertical="GENERIC",
    solution_types=["SOLUTION_TYPE_SEARCH"],
    content_config="CONTENT_REQUIRED",
)

create_data_store_request = vais.CreateDataStoreRequest(
    parent=f"projects/{PROJECT_ID}/locations/{VAIS_LOCATION}/collections/default_collection",
    data_store=data_store,
    data_store_id=DATA_STORE_ID,
)

# The api returns long running operation as response.
create_data_store_operation = data_store_service_client.create_data_store(
    create_data_store_request
)
print(
    f"Waiting for create datastore operation to complete: {create_data_store_operation.operation.name}"
)
created_data_store = create_data_store_operation.result()
data_store_name = created_data_store.name
print(f"Data store {data_store_name} is created.")

In [ ]:
# @title Ingest documents

# fmt: off
GCS_SOURCE = "gs://cloud-samples-data/gen-app-builder/search/cymbal-bank-employee"  # @param {type:"string"}
# fmt: on

branch_path = document_service_client.branch_path(
    project=PROJECT_ID,
    location=VAIS_LOCATION,
    data_store=DATA_STORE_ID,
    branch="default_branch",
)

document_service_client.import_documents(
    request=vais.ImportDocumentsRequest(
        parent=branch_path,
        gcs_source=vais.GcsSource(
            input_uris=[f"{GCS_SOURCE}/*"], data_schema="content"
        ),
        reconciliation_mode=vais.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
    )
)

In [ ]:
# @title Wait for documents imported

document_imported = False
while not document_imported:
    for doc in document_service_client.list_documents(parent=branch_path):
        print(doc.content.uri)
        document_imported = True
    time.sleep(10)

In [ ]:
# @title Create engine

engine_id = f"{DATA_STORE_ID}_engine"
create_engine_request = vais.CreateEngineRequest(
    parent=engine_client.collection_path(
        project=PROJECT_ID, location=VAIS_LOCATION, collection="default_collection"
    ),
    engine=vais.Engine(
        display_name="Engine for Vertex LLM Grounding demo",
        solution_type=vais.SolutionType.SOLUTION_TYPE_SEARCH,
        search_engine_config=vais.Engine.SearchEngineConfig(
            search_tier=vais.SearchTier.SEARCH_TIER_ENTERPRISE,
            search_add_ons=[vais.SearchAddOn.SEARCH_ADD_ON_LLM],
        ),
        common_config=vais.Engine.CommonConfig(company_name="Cymbal"),
        industry_vertical=vais.IndustryVertical.GENERIC,
        data_store_ids=[DATA_STORE_ID],
    ),
    engine_id=engine_id,
)
create_engine_operation = engine_client.create_engine(create_engine_request)
wait_for_operation_finish(create_engine_operation)
print("Successfully create engine")

Give it a few minutes to build the index. To verify the engine is ready to use, you may try a search request against in. For example:

In [ ]:
search_client = vais.SearchServiceClient(client_options=client_options)

while True:
    try:
        search_client.search(
            vais.SearchRequest(
                serving_config=(
                    f"projects/{PROJECT_ID}/locations/{VAIS_LOCATION}/collections/default_collection/engines/{engine_id}/servingConfigs/default_search"
                ),
                query="Who is the CEO?",
                page_size=10,
            )
        )
    except Exception:
        time.sleep(10)

## Calling Vertex AI LLM grounded on your Vertex AI Search engine

Once your engine is ready, you can use it as a grounding source in a Vertex AI LLM call, as shown below:

In [ ]:
# @title Initialize Gen AI client

from IPython.display import Markdown, display
from google import genai
from google.genai.types import GenerateContentConfig, Retrieval, Tool, VertexAISearch

# fmt: off
MODEL_ID = "gemini-2.0-flash"  # @param {type: "string"}
# fmt: on


client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

vais_tool = Tool(
    retrieval=Retrieval(
        vertex_ai_search=VertexAISearch(
            engine=f"projects/{PROJECT_ID}/locations/global/collections/default_collection/engines/{engine_id}",
        )
    )
)

In [ ]:
# fmt: off
PROMPT = "Instruct me how to book the flight for my business trip"  # @param {type: "string"}
# fmt: on

response = client.models.generate_content(
    model=MODEL_ID,
    contents=PROMPT,
    config=GenerateContentConfig(tools=[vais_tool]),
)

if response.candidates:
    display(Markdown(response.text))

Behind the scenes, the model will send a search query to the Vertex AI engine to retrieve information that helps it generate a response. If the generated text makes a claim strongly based on the retrieved result, the model will also output that correlation under grounding metadata. For example, you can see the supporting documents for specific claims in the generated output:

In [ ]:
if response.candidates[0].grounding_metadata.grounding_supports:
    for s in response.candidates[0].grounding_metadata.grounding_supports:
        display(Markdown(f"{s.segment.text} {s.grounding_chunk_indices}"))

And to see all retrieved results:

In [ ]:
for i, chunk in enumerate(response.candidates[0].grounding_metadata.grounding_chunks):
    print(f"{i}.\n")
    display(Markdown(chunk.retrieved_context.text))
    print(chunk.retrieved_context.uri)
    print("\n")

Note: The model may not always output grounding support, even with successful retrieval. This occurs when the model doesn't find a strong enough corroboration in the retrieved information.

## Cleaning up

Delete `engine` and `dataStore` created in this tutorial

In [ ]:
# Delete engine
engine_full_resource_path = engine_client.engine_path(
    project=PROJECT_ID,
    location=VAIS_LOCATION,
    collection="default_collection",
    engine=engine_id,
)

delete_engine_operation = engine_client.delete_engine(engine_full_resource_path)
print(
    f"Waiting for delete engine operation to complete: {delete_engine_operation.operation.name}"
)
wait_for_operation_finish(delete_engine_operation)
print(f"Successfully deleted engine {engine_full_resource_path}")

In [ ]:
# Delete the data store
data_store_full_resource_path = data_store_service_client.data_store_path(
    project=PROJECT_ID, location=VAIS_LOCATION, data_store=DATA_STORE_ID
)
delete_data_store_operation = data_store_service_client.delete_data_store(
    name=data_store_full_resource_path
)
print(
    f"Waiting for delete data store operation to complete: {delete_data_store_operation.operation.name}"
)
wait_for_operation_finish(delete_data_store_operation)
print(f"Successfully deleted data store {data_store_full_resource_path}")